In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
import shap
import xgboost as xgb
import matplotlib.pyplot as plt
from typing import Union, Tuple, Optional
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 1. ОЧИСТКА И КОНСОЛИДАЦИЯ ДАННЫХ
def load_and_clean_data(file_path):
    """
    Загрузка и очистка данных из CSV файла
    """
    # Загрузка данных
    df = pd.read_csv(file_path, parse_dates=['Дата счёта'])
    
    # Переименование колонок для удобства
    column_mapping = {
        'Клиент': 'client_id',
        'Область': 'region',
        'SKU': 'sku',
        'Дата счёта': 'invoice_date',
        'Количество (шт.)': 'quantity',
        'Цена (р.)': 'price',
        'Сумма_заказа': 'final_price',
        'Группа': 'group',
        'Тип': 'type',
        'Категория': 'category'
    }
    
    # Применяем переименование для существующих колонок
    df.rename(columns={k: v for k, v in column_mapping.items() if k in df.columns}, inplace=True)
    
    # Обработка числовых колонок
    numeric_columns = ['quantity', 'price', 'final_price']
    for col in numeric_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Заполнение пропущенных значений
    df.fillna({
        'quantity': 0,
        'price': 0,
        'final_price': 0
    }, inplace=True)
    
    
    # Добавление временных компонентов для анализа
    if 'invoice_date' in df.columns:
        df['year'] = df['invoice_date'].dt.year
        df['month'] = df['invoice_date'].dt.month
        df['quarter'] = df['invoice_date'].dt.quarter
        df['date_key'] = df['invoice_date'].dt.to_period('M').dt.start_time
    
    return df
    
def filter_and_deduplicate_canceled_orders(data, time_window='D'):
    """
    Фильтрует отмененные заказы и удаляет дубликаты в рамках заданного временного окна
    
    Параметры:
    data (DataFrame): Датафрейм с данными о заказах
    time_window (str): Временное окно для группировки ('D' для дня)
    
    Возвращает:
    DataFrame: Датафрейм с уникальными отмененными заказами
    """
    # Копируем данные, чтобы избежать изменения исходного датафрейма
    df = data.copy()
    
    # Фильтруем отмененные заказы, если есть соответствующая колонка
    if 'order_status' in df.columns:
        df = df[df['order_status'] == 'canceled']
    
    # Убедимся, что дата в правильном формате
    date_column = 'Дата счёта' if 'Дата счёта' in df.columns else 'invoice_date'
    
    if not pd.api.types.is_datetime64_any_dtype(df[date_column]):
        df[date_column] = pd.to_datetime(df[date_column])
    
    # Создаем временное окно для группировки
    df['time_window'] = df[date_column].dt.floor(time_window)
    
    # Определяем ключевые колонки для поиска дубликатов
    client_col = 'Клиент' if 'Клиент' in df.columns else 'client_id'
    sku_col = 'SKU' if 'SKU' in df.columns else 'sku'
    quantity_col = 'Количество' if 'Количество' in df.columns else 'quantity'
    price_col = 'Цена (р.)' if 'Цена (р.)' in df.columns else 'price'
    
    # Удаляем дубликаты, оставляя только одно значение для каждой комбинации
    deduplicated = df.drop_duplicates(
        subset=[client_col, sku_col, quantity_col, price_col, 'time_window']
    )
    
    # Удаляем временную колонку
    deduplicated = deduplicated.drop(columns=['time_window'])
    
    return deduplicated

def consolidate_orders(completed_orders_path, uncompleted_orders_path):
    """
    Консолидация выполненных и невыполненных заказов
    """
    # Загрузка и очистка данных
    completed = load_and_clean_data(completed_orders_path)
    uncompleted = load_and_clean_data(uncompleted_orders_path)
    uncompleted = filter_and_deduplicate_canceled_orders(uncompleted)
    # Добавление флага статуса заказа
    completed['order_status'] = 'completed'
    uncompleted['order_status'] = 'uncompleted'
    
    # Консолидация данных
    consolidated = pd.concat([completed, uncompleted], sort=False)
    consolidated.fillna(0)
    consolidated = consolidated.replace('UNK', 'Unk')
    consolidated[consolidated['order_status']=='uncompleted']['final_price'] = 0
    return consolidated

In [3]:
def prepare_time_series(data, freq='M'):
    """
    Подготовка временных рядов для прогнозирования с сохранением дополнительных агрегированных столбцов 
    """
    # Копируем данные, чтобы избежать изменений во входных данных
    data_copy = data.copy()
    
    # Группировка данных по дате
    if 'order_status' in data_copy.columns:
        data_copy = data_copy[data_copy['order_status'] == 'completed']
    
    # Убедимся, что у нас есть колонка с датой
    if 'date_key' not in data_copy.columns and 'invoice_date' in data_copy.columns:
        data_copy['date_key'] = data_copy['invoice_date'].dt.to_period(freq).dt.start_time
    
    # Добавляем расчет недели месяца
    data_copy['week_of_month'] = ((data_copy['invoice_date'].dt.day - 1) // 7) + 1
    
    # Функция для нахождения наиболее частого значения
    def most_frequent(series):
        if series.empty:
            return None
        counts = series.value_counts()
        if counts.empty:
            return None
        return counts.index[0]
    
    # Создадим словарь агрегаций
    agg_dict = {}
    
    # Добавляем базовые агрегации если соответствующие столбцы существуют
    if 'quantity' in data_copy.columns:
        agg_dict['quantity'] = 'sum'
    if 'final_price' in data_copy.columns:
        agg_dict['final_price'] = 'sum'
    
    # Определяем типы столбцов и добавляем соответствующие агрегации
    for col in data_copy.columns:
        # Пропускаем уже обработанные столбцы и служебные
        if col in agg_dict or col in ['date_key', 'order_status', 'invoice_date', 'Unnamed: 0', 
                                      'Дата год-месяц', 'client_id', 'sku', 'time_idx', 
                                      'days_until_holiday', 'is_holiday', 'day_of_week', 'week_of_month']:
            continue
        if col not in ['usd_rub_Мин', 'usd_rub_Откр', 'usd_rub_Цена', 'inflation_rate']:
            # Определяем тип данных и выбираем подходящую агрегацию
            dtype = data_copy[col].dtype
            if pd.api.types.is_numeric_dtype(dtype):  # Для числовых колонок
                agg_dict[col] = 'max'
            elif pd.api.types.is_object_dtype(dtype) or pd.api.types.is_categorical_dtype(dtype):  # Для категориальных
                agg_dict[col] = most_frequent
            elif pd.api.types.is_datetime64_dtype(dtype):  # Для дат
                agg_dict[col] = 'max'
            elif pd.api.types.is_bool_dtype(dtype):  # Для булевых
                agg_dict[col] = 'any'
                
        elif col == 'usd_rub_Мин':
            agg_dict[col] = 'min'
            
        elif col == 'usd_rub_Откр':
            agg_dict[col] = 'mean'

        elif col == 'inflation_rate':
            agg_dict[col] = 'mean'

    # Вычисляем суммарную прибыль по неделям месяца
    weekly_sums = data_copy.groupby(['date_key', 'week_of_month'])['final_price'].sum().reset_index()
    
    # Преобразуем в широкий формат
    weekly_pivot = weekly_sums.pivot(
        index='date_key', 
        columns='week_of_month', 
        values='final_price'
    ).fillna(0)
    
    #переименовываем числовые столбцы в строковые
    weekly_pivot = weekly_pivot.rename(columns={
        1: 'week_1_revenue',
        2: 'week_2_revenue',
        3: 'week_3_revenue',
        4: 'week_4_revenue',
        5: 'week_5_revenue'
    })
    
    # Группировка данных по периоду времени с учетом всех агрегаций
    time_series = data_copy.groupby('date_key').agg(agg_dict).reset_index()
    time_series = time_series.merge(weekly_pivot, on='date_key', how='left')
    time_series = time_series.fillna(0)
    
    # Установка даты в качестве индекса
    time_series.set_index('date_key', inplace=True)
    
    # Убедимся, что индекс отсортирован
    time_series = time_series.sort_index()
    return time_series


In [4]:
# 2. РАСЧЕТ ПОТЕНЦИАЛЬНЫХ ПРОДАЖ
def calculate_potential_sales(consolidated_data):
    """
    Расчет потенциальных продаж на основе фактических и упущенных заказов
    """
    # Определяем измерения для группировки
    dimensions = ['sku', 'year', 'month', 'quarter', 'region', 'group', 'category']
    existing_dims = [d for d in dimensions if d in consolidated_data.columns]
    
    # Расчет фактических продаж
    completed_sales = consolidated_data[consolidated_data['order_status'] == 'completed'].groupby(
        existing_dims
    ).agg({
        'quantity': 'sum',
        'final_price': 'sum',
        'price': 'mean'
    }).reset_index()
    completed_sales.rename(columns={'final_price': 'actual_revenue'}, inplace=True)
    
    # Расчет упущенных продаж
    lost_sales = consolidated_data[consolidated_data['order_status'] == 'uncompleted'].groupby(
        existing_dims
    ).agg({
        'quantity': 'sum'
    }).reset_index()
    lost_sales.rename(columns={'quantity': 'lost_quantity'}, inplace=True)
    
    # Объединение данных
    potential_data = pd.merge(
        completed_sales, lost_sales, 
        on=existing_dims, 
        how='outer'
    ).fillna(0)
    
    # Расчет потенциальной выручки
    potential_data['lost_revenue'] = potential_data['lost_quantity'] * potential_data['price']
    potential_data['potential_revenue'] = potential_data['actual_revenue'] + potential_data['lost_revenue']
    potential_data['loss_ratio'] = potential_data['lost_revenue'] / potential_data['potential_revenue'].replace(0, np.nan)
    
    return potential_data

In [5]:
def forecast_by_segment(data, segment_column, periods=12, methods = "xgboost"):
    """
    Прогнозирование продаж по сегментам (категория, группа, регион и т.д.)
    """
    # Проверка наличия колонки сегмента
    if segment_column not in data.columns:
        raise ValueError(f"Колонка '{segment_column}' не найдена в данных")
    
    # Получение уникальных значений сегмента
    segments = data[segment_column].unique()
    
    # Словарь для хранения прогнозов
    forecasts = {}
    
    # Прогнозирование для каждого сегмента
    for segment in segments:
        segment_data = data[data[segment_column] == segment]
        
        # Пропускаем сегменты с недостаточным количеством данных
        if len(segment_data) < 12:
            continue
        
        # Подготовка временного ряда
        ts = prepare_time_series(segment_data)
        
        # Прогнозирование
        forecast = forecast_sales(ts, method = methods, periods=periods)
        
        # Сохранение результата
        forecasts[segment] = forecast
    
    return forecasts

In [6]:

# 4. АНАЛИТИЧЕСКИЕ ОТЧЕТЫ ПО ТРЕНДАМ
def generate_trend_report(data, dimensions, metrics, time_unit='month'):
    """
    Генерация аналитических отчетов по трендам по разным измерениям
    """
    # Подготовка данных с временным ключом
    if 'date_key' not in data.columns and 'invoice_date' in data.columns:
        if time_unit == 'month':
            data['date_key'] = data['invoice_date'].dt.to_period('M').dt.start_time
        elif time_unit == 'quarter':
            data['date_key'] = data['invoice_date'].dt.to_period('Q').dt.start_time
        elif time_unit == 'year':
            data['date_key'] = data['invoice_date'].dt.to_period('Y').dt.start_time
    
    # Убедимся, что dimensions и metrics - списки
    if isinstance(dimensions, str):
        dimensions = [dimensions]
    if isinstance(metrics, str):
        metrics = [metrics]
    
    # Словарь для хранения отчетов
    reports = {}
    
    # Генерация отчетов для каждого измерения
    for dimension in dimensions:
        # Проверка наличия измерения в данных
        if dimension not in data.columns:
            continue
        
        # Получение уникальных значений измерения
        values = data[dimension].unique()
        
        # Словарь для хранения трендов по значениям измерения
        dimension_trends = {}
        
        # Анализ тренда для каждого значения измерения
        for value in values:
            value_data = data[data[dimension] == value]
            
            # Группировка по времени и расчет метрик
            trend = value_data.groupby('date_key').agg({
                metric: 'sum' for metric in metrics if metric in data.columns
            }).reset_index()
            
            # Сортировка по времени
            trend = trend.sort_values('date_key')
            
            # Сохранение тренда
            dimension_trends[value] = trend
        
        # Сохранение отчета по измерению
        reports[dimension] = dimension_trends
    
    return reports

In [7]:
def get_top_performers(data, dimension, metric, n=10, ascending=False):
    """
    Получение топ-N значений по заданному измерению и метрике
    """
    # Проверка наличия колонок
    if dimension not in data.columns or metric not in data.columns:
        raise ValueError(f"Колонки '{dimension}' или '{metric}' не найдены в данных")
    
    # Группировка и расчет суммы метрики
    summary = data.groupby(dimension).agg({
        metric: 'sum'
    }).reset_index()
    
    # Сортировка и выбор топ-N
    top_n = summary.sort_values(metric, ascending=ascending).head(n)
    
    return top_n

In [8]:

def get_feature_importance(
    model: Union[xgb.XGBModel, xgb.Booster],
    X: Union[np.ndarray, pd.DataFrame],
    importance_type: str = 'gain',
    return_shap_values: bool = False,
    plot: bool = False,
    figsize: Tuple[int, int] = (12, 8)
) -> Union[pd.DataFrame, Tuple[pd.DataFrame, np.ndarray]]:
    """
    Получает важность признаков из модели XGBoost, используя как встроенные методы, так и SHAP.
    
    Параметры:
    -----------
    model : Union[xgb.XGBModel, xgb.Booster]
        Обученная модель XGBoost.
    X : Union[np.ndarray, pd.DataFrame]
        Данные, на которых модель была обучена или для которых нужно рассчитать важность.
    importance_type : str, по умолчанию 'gain'
        Тип важности для XGBoost (weight, gain, cover, total_gain, total_cover).
    return_shap_values : bool, по умолчанию False
        Если True, то возвращает детальные значения SHAP вместе с DataFrame.
    plot : bool, по умолчанию False
        Если True, то создает визуализацию важности признаков.
    figsize : Tuple[int, int], по умолчанию (12, 8)
        Размер графика, если plot=True.
        
    Возвращает:
    -----------
    pd.DataFrame или Tuple[pd.DataFrame, np.ndarray]
        DataFrame с важностью признаков и опционально массив со значениями SHAP.
    """
    # Определяем имена признаков
    if isinstance(X, pd.DataFrame):
        feature_names = X.columns.tolist()
    else:
        feature_names = [f'feature_{i}' for i in range(X.shape[1])]
    
    # Получаем важность признаков из XGBoost
    if isinstance(model, xgb.Booster):
        xgb_importance_dict = model.get_score(importance_type=importance_type)
        # Не все признаки могут быть в словаре, если они не использовались
        xgb_importance = np.zeros(len(feature_names))
        for feature, importance in xgb_importance_dict.items():
            # Предполагаем, что признаки в XGBoost имеют формат 'f0', 'f1', ... или соответствуют именам
            if feature.startswith('f') and feature[1:].isdigit():
                index = int(feature[1:])
                if index < len(xgb_importance):
                    xgb_importance[index] = importance
            elif feature in feature_names:
                index = feature_names.index(feature)
                xgb_importance[index] = importance
    else:  # для scikit-learn API (XGBRegressor, XGBClassifier)
        xgb_importance = model.feature_importances_
    
    # Создаем DataFrame для XGBoost importance
    xgb_importance_df = pd.DataFrame({
        'Feature': feature_names,
        f'XGB_{importance_type.capitalize()}_Importance': xgb_importance
    })
    
    # Получаем SHAP values
    try:
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(X)
        
        # Обрабатываем разные форматы SHAP values
        if isinstance(shap_values, list):  # для мультиклассовой классификации
            shap_importance = np.mean([np.abs(sv).mean(axis=0) for sv in shap_values], axis=0)
        else:  # для регрессии или бинарной классификации
            shap_importance = np.abs(shap_values).mean(axis=0)
        
        # Создаем DataFrame для SHAP importance
        shap_importance_df = pd.DataFrame({
            'Feature': feature_names,
            'SHAP_Importance': shap_importance
        })
        
        # Объединяем оба DataFrame
        combined_importance = pd.merge(shap_importance_df, xgb_importance_df, on='Feature')
        
        # Сортируем по SHAP importance
        combined_importance = combined_importance.sort_values('SHAP_Importance', ascending=False)
        
        # Визуализация, если требуется
        if plot:
            plt.figure(figsize=figsize)
            
            # Создаем два подграфика
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
            
            # Строим график для SHAP importance
            combined_importance.sort_values('SHAP_Importance', ascending=True).plot.barh(
                x='Feature', y='SHAP_Importance', ax=ax1
            )
            ax1.set_title('SHAP Importance')
            
            # Строим график для XGBoost importance
            combined_importance.sort_values(f'XGB_{importance_type.capitalize()}_Importance', ascending=True).plot.barh(
                x='Feature', y=f'XGB_{importance_type.capitalize()}_Importance', ax=ax2
            )
            ax2.set_title(f'XGBoost {importance_type.capitalize()} Importance')
            
            plt.tight_layout()
            plt.show()
            
            # Дополнительно добавляем SHAP summary plot, если требуется visualization
            shap.summary_plot(shap_values, X if isinstance(X, pd.DataFrame) else pd.DataFrame(X, columns=feature_names), 
                             plot_type="bar", show=False)
            plt.title('SHAP Feature Importance')
            plt.tight_layout()
            plt.show()
        
        if return_shap_values:
            return combined_importance, shap_values
        else:
            return combined_importance
            
    except Exception as e:
        print(f"Произошла ошибка при расчете SHAP values: {e}")
        # Если SHAP не удался, возвращаем только XGBoost importance
        xgb_importance_df = xgb_importance_df.sort_values(
            f'XGB_{importance_type.capitalize()}_Importance', ascending=False
        )
        return xgb_importance_df


In [9]:
def forecast_sales(time_series, periods=12, method='prophet', exog_vars=None):
    """
    Прогнозирование продаж на будущий период используя современные методы машинного обучения.
    
    Параметры:
    time_series : DataFrame - временной ряд с колонками 'quantity' и 'final_price'
    periods : int - количество периодов для прогноза
    method : str - метод прогнозирования ('prophet', 'xgboost', 'average')
    exog_vars : DataFrame - экзогенные переменные для моделей (опционально)
    
    Возвращает:
    DataFrame с историческими данными и прогнозом
    """
    import pandas as pd
    import numpy as np
    from datetime import datetime
    import warnings
    warnings.filterwarnings('ignore')
    
    # Проверка наличия данных в временном ряде
    if time_series.empty:
        print("Временной ряд пуст. Прогнозирование невозможно.")
        return pd.DataFrame()  # Возвращаем пустой DataFrame

    # Если недостаточно данных для сложных моделей, используем простой метод
    if len(time_series) < 12:
        print("Недостаточно данных для сложных моделей. Используем метод prophet.")
        method = 'prophet'
        
    # Определение последней даты и создание дат для прогноза
    try:
        last_date = time_series.index[-1]
        forecast_dates = [last_date + pd.DateOffset(months=i+1) for i in range(periods)]
    except Exception as e:
        print(f"Ошибка при работе с индексом дат: {e}")
        return pd.DataFrame()

    # Прогнозирование методом Prophet
    if method == 'prophet':
        try:
            from prophet import Prophet
            from sklearn.preprocessing import LabelEncoder
            
            # Создаем словарь для хранения энкодеров
            encoders = {}
            
            # Подготовка данных для Prophet
            df_prophet_quantity = pd.DataFrame({
                'ds': time_series.index,
                'y': time_series['quantity'].values
            })
            
            df_prophet_revenue = pd.DataFrame({
                'ds': time_series.index,
                'y': time_series['final_price'].values
            })
            
            # Создание моделей Prophet с логистическим ростом для предотвращения отрицательных значений
            model_quantity = Prophet(
                growth='logistic',
                seasonality_mode='multiplicative',
                yearly_seasonality=True, 
                weekly_seasonality=False, 
                daily_seasonality=False
            )
            
            model_revenue = Prophet(
                growth='logistic',
                seasonality_mode='multiplicative',
                yearly_seasonality=True, 
                weekly_seasonality=False, 
                daily_seasonality=False
            )
            
            # Установка нижней и верхней границы для логистического роста
            df_prophet_quantity['floor'] = 0
            df_prophet_quantity['cap'] = df_prophet_quantity['y'].max() * 1.5
            
            df_prophet_revenue['floor'] = 0
            df_prophet_revenue['cap'] = df_prophet_revenue['y'].max() * 1.5
            
            # Обработка числовых и категориальных регрессоров
            for col in time_series.columns:
                # Пропускаем целевые переменные
                if col not in ['quantity', 'final_price']:
                    # Преобразуем имя колонки в строку
                    col_name = str(col)
                    
                    # Проверяем, является ли столбец числовым или категориальным
                    if pd.api.types.is_numeric_dtype(time_series[col]):
                        # Для числовых столбцов добавляем как есть
                        df_prophet_quantity[col_name] = time_series[col].values
                        df_prophet_revenue[col_name] = time_series[col].values
                        model_quantity.add_regressor(col_name)
                        model_revenue.add_regressor(col_name)
                    elif pd.api.types.is_categorical_dtype(time_series[col]) or pd.api.types.is_object_dtype(time_series[col]):
                        # Для категориальных создаем и сохраняем энкодер
                        encoders[col] = LabelEncoder()
                        # Преобразуем в строки перед кодированием для безопасности
                        encoded_values = encoders[col].fit_transform(time_series[col].astype(str))
                        df_prophet_quantity[col_name] = encoded_values
                        df_prophet_revenue[col_name] = encoded_values
                        model_quantity.add_regressor(col_name)
                        model_revenue.add_regressor(col_name)
    
            # Добавление экзогенных переменных
            if exog_vars is not None:
                for col in exog_vars.columns:
                    if col in df_prophet_quantity.columns:
                        continue
                        
                    if pd.api.types.is_numeric_dtype(exog_vars[col]):
                        df_prophet_quantity[col] = exog_vars[col].values
                        df_prophet_revenue[col] = exog_vars[col].values
                    else:
                        # Кодирование категориальных переменных
                        encoders[col] = LabelEncoder()
                        encoded_values = encoders[col].fit_transform(exog_vars[col].astype(str))
                        df_prophet_quantity[col] = encoded_values
                        df_prophet_revenue[col] = encoded_values
                        
                    model_quantity.add_regressor(col)
                    model_revenue.add_regressor(col)
            
            # Обучение моделей
            model_quantity.fit(df_prophet_quantity)
            model_revenue.fit(df_prophet_revenue)
            
            # Создание фрейма для прогноза
            future_quantity = model_quantity.make_future_dataframe(periods=periods, freq='M')
            future_revenue = model_revenue.make_future_dataframe(periods=periods, freq='M')
            
            # Добавляем границы для логистического роста
            future_quantity['floor'] = 0
            future_quantity['cap'] = df_prophet_quantity['cap'].max()
            
            future_revenue['floor'] = 0
            future_revenue['cap'] = df_prophet_revenue['cap'].max()
            
            # Создаем копии закодированных признаков
            encoded_data = pd.DataFrame(index=time_series.index)
            
            # Для каждой колонки сохраняем оригинальные и закодированные значения
            for col in time_series.columns:
                if col not in ['quantity', 'final_price']:
                    if col in encoders:
                        # Сохраняем закодированные значения
                        encoded_data[col] = df_prophet_quantity[col].values
                    else:
                        # Сохраняем оригинальные значения для числовых признаков
                        encoded_data[col] = time_series[col].values
            
            # Объединяем с будущими датафреймами
            future_quantity = pd.merge(
                future_quantity, 
                encoded_data, 
                left_on='ds', 
                right_index=True, 
                how='left'
            )
            
            future_revenue = pd.merge(
                future_revenue, 
                encoded_data, 
                left_on='ds', 
                right_index=True, 
                how='left'
            )
            
            # Определяем, какие строки относятся к будущим датам (те, где будут NaN)
            is_future_date = future_quantity['ds'].apply(
                lambda x: x not in time_series.index
            )
            future_indices = future_quantity[is_future_date].index
            
            # Заполняем пропуски (будущие даты) последними известными значениями
            for col in encoded_data.columns:
                # Проверяем наличие NaN
                if future_quantity[col].isna().any():
                    # Берем последнее известное значение
                    last_value = encoded_data[col].iloc[-1]
                    # Заполняем только для будущих дат
                    future_quantity.loc[future_indices, col] = last_value
                    future_revenue.loc[future_indices, col] = last_value
            
            # Проверка наличия NaN перед прогнозированием
            nan_columns = future_quantity.columns[future_quantity.isna().any()].tolist()
            if nan_columns:
                print(f"Обнаружены NaN в следующих столбцах: {nan_columns}")
                # Автоматическое заполнение пропущенных значений
                for col in nan_columns:
                    # Используем среднее или моду в зависимости от типа данных
                    if pd.api.types.is_numeric_dtype(future_quantity[col]):
                        fill_value = future_quantity[col].mean()
                    else:
                        # Для нечисловых данных используем наиболее частое значение
                        non_na_values = future_quantity[col].dropna()
                        fill_value = non_na_values.value_counts().idxmax() if not non_na_values.empty else 0
                    
                    future_quantity[col] = future_quantity[col].fillna(fill_value)
                    future_revenue[col] = future_revenue[col].fillna(fill_value)
            
            # Добавление экзогенных переменных для прогноза
            if exog_vars is not None:
                for col in exog_vars.columns:
                    if col in future_quantity.columns and col not in encoders:
                        continue
                    
                    # Заполняем будущие значения из exog_vars при наличии
                    if len(exog_vars) >= periods:
                        future_values = exog_vars[col].iloc[-periods:].values
                    else:
                        future_values = list(exog_vars[col].values) * (periods // len(exog_vars) + 1)
                        future_values = future_values[:periods]
                    
                    # Если колонка категориальная, применяем тот же энкодер
                    if col in encoders:
                        future_values = encoders[col].transform([str(x) for x in future_values])
                    
                    # Заполняем только будущие даты (future_indices)
                    for i, idx in enumerate(future_indices):
                        if i < len(future_values):
                            future_quantity.loc[idx, col] = future_values[i]
                            future_revenue.loc[idx, col] = future_values[i]
            
            # Окончательная проверка на NaN перед прогнозом
            for df in [future_quantity, future_revenue]:
                for col in df.columns:
                    if df[col].isna().any():
                        print(f"Предупреждение: колонка {col} все еще содержит NaN")
                        # Заполняем нулями в крайнем случае
                        df[col] = df[col].fillna(0)
            
            # Прогнозирование
            forecast_quantity = model_quantity.predict(future_quantity)
            forecast_revenue = model_revenue.predict(future_revenue)
            
            # Создание датафрейма прогноза
            forecast_df = pd.DataFrame({
                'forecast_quantity':[abs(q) for q in forecast_quantity.tail(periods)['yhat'].values],
                'forecast_revenue': [abs(r) for r in forecast_revenue.tail(periods)['yhat'].values],
                'quantity_lower': [abs(q)*0.85 for q in forecast_quantity.tail(periods)['yhat_lower'].values],
                'quantity_upper': [abs(q)*1.15 for q in forecast_quantity.tail(periods)['yhat_upper'].values],
                'revenue_lower': [abs(r)*0.85 for r in forecast_revenue.tail(periods)['yhat_lower'].values],
                'revenue_upper': [abs(r)*1.15 for r in forecast_revenue.tail(periods)['yhat_upper'].values]
            }, index=forecast_dates)
            
        except Exception as e:
            print(f"Ошибка при прогнозировании методом Prophet: {e}")
            import traceback
            traceback.print_exc()  # Полный стек ошибки для отладки
            return pd.DataFrame()
    
       # Прогнозирование методом XGBoost
    elif method == 'xgboost':
        try:
            import shap
            import xgboost as xgb
            import numpy as np
            from sklearn.preprocessing import StandardScaler, LabelEncoder
            
            # Создаем копию данных для безопасности
            data_copy = time_series.copy()
            
            # Применяем логарифмическое преобразование к целевым переменным
            # Добавляем 1, чтобы избежать log(0)
            data_copy['quantity_log'] = np.log1p(data_copy['quantity'])
            data_copy['final_price_log'] = np.log1p(data_copy['final_price'])
            
            # Кодирование категориальных переменных
            encoders = {}
            for col in data_copy.columns:
                if col not in ['quantity', 'final_price', 'quantity_log', 'final_price_log']:
                    if not pd.api.types.is_numeric_dtype(data_copy[col]):
                        encoders[col] = LabelEncoder()
                        data_copy[col] = encoders[col].fit_transform(data_copy[col].astype(str))
            
            # Функция для создания признаков
            def create_features(df, label=None, lags=None, ensure_features=None):
                df_new = df.copy()
                
                # Создание лагов
                if lags is None:
                    lags = [1, 2, 3, 6, 12]
                
                # Используем логарифмированные значения для лагов
                for lag in lags:
                    df_new[f'quantity_lag_{lag}'] = df_new['quantity_log'].shift(lag)
                    df_new[f'final_price_lag_{lag}'] = df_new['final_price_log'].shift(lag)
                
                # Добавление временных признаков
                df_new['month'] = df_new.index.month
                df_new['quarter'] = df_new.index.quarter
                df_new['year'] = df_new.index.year
                
                # Добавление скользящего среднего для логарифмированных значений
                for window in [3, 6, 12]:
                    df_new[f'quantity_rolling_{window}'] = df_new['quantity_log'].rolling(window=window).mean()
                    df_new[f'final_price_rolling_{window}'] = df_new['final_price_log'].rolling(window=window).mean()
                
                # Добавление экзогенных переменных
                if exog_vars is not None:
                    for col in exog_vars.columns:
                        if not pd.api.types.is_numeric_dtype(exog_vars[col]):
                            if col in encoders:
                                values = encoders[col].transform(exog_vars[col].astype(str))
                            else:
                                encoder = LabelEncoder()
                                values = encoder.fit_transform(exog_vars[col].astype(str))
                                encoders[col] = encoder
                            df_new[col] = values
                        else:
                            df_new[col] = exog_vars[col].values
                
                # Проверка наличия всех необходимых признаков
                if ensure_features is not None:
                    for feature in ensure_features:
                        if feature not in df_new.columns and feature != label:
                            df_new[feature] = 0
                
                # Заполнение пропусков и получение признаков
                df_new = df_new.fillna(df_new.mean())
                all_features = [col for col in df_new.columns if col != label and 
                               col not in ['quantity', 'final_price']]
                
                if label and label in df_new.columns:
                    X = df_new[all_features]
                    y = df_new[label]
                else:
                    X = df_new
                    y = None
                
                return X, y, all_features
            
            # Создание признаков для обучения с логарифмированными целевыми переменными
            X_quantity, y_quantity, feature_list_quantity = create_features(data_copy, label='quantity_log')
            X_revenue, y_revenue, feature_list_revenue = create_features(data_copy, label='final_price_log')
            
            # Преобразование в DataFrame
            X_quantity = pd.DataFrame(
                X_quantity,
                columns=X_quantity.columns,
                index=X_quantity.index
            )
            
            X_revenue = pd.DataFrame(
                X_revenue,
                columns=X_revenue.columns,
                index=X_revenue.index
            )
            
            # Обучение моделей с стандартной функцией
            model_quantity = xgb.XGBRegressor(
                objective='reg:squarederror',  # Стандартная функция потерь
                n_estimators=1000,
                learning_rate=0.05,
                max_depth=5,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42
            )
            
            model_revenue = xgb.XGBRegressor(
                objective='reg:squarederror',  # Стандартная функция потерь
                n_estimators=1000,
                learning_rate=0.05,
                max_depth=5,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42
            )
            
            model_quantity.fit(X_quantity, y_quantity)
            model_revenue.fit(X_revenue, y_revenue)
            
            #Получение feature_importans and SHAP
            quantity_importance_df = get_feature_importance(model_quantity, X_quantity)
            revenue_importance_df = get_feature_importance(model_revenue, X_revenue)
            quantity_importance_df.to_csv('quantity_importance_df.csv')
            revenue_importance_df.to_csv('revenue_importance_df.csv')
            
            # Рекурсивное прогнозирование
            forecast_quantity = []
            forecast_revenue = []
            
            # Копируем последние данные для прогноза
            future_data = data_copy.copy().iloc[-12:] if len(data_copy) >= 12 else data_copy.copy()
            
            # Итеративное прогнозирование
            for i in range(periods):
                next_date = last_date + pd.DateOffset(months=i+1)
                
                # Создаем признаки для прогноза
                X_future_quantity, _, _ = create_features(
                    future_data, 
                    ensure_features=feature_list_quantity
                )
                
                X_future_revenue, _, _ = create_features(
                    future_data, 
                    ensure_features=feature_list_revenue
                )
                
                # Проверка наличия всех признаков
                X_future_quantity = X_future_quantity[feature_list_quantity]
                X_future_revenue = X_future_revenue[feature_list_revenue]
                
                # Преобразование в DataFrame
                X_future_quantity = pd.DataFrame(
                    X_future_quantity,
                    columns=X_future_quantity.columns,
                    index=X_future_quantity.index
                )
                
                X_future_revenue = pd.DataFrame(
                    X_future_revenue,
                    columns=X_future_revenue.columns,
                    index=X_future_revenue.index
                )
                
                # Прогнозирование в логарифмической шкале
                log_quantity_pred = model_quantity.predict(X_future_quantity.iloc[-1:])
                log_revenue_pred = model_revenue.predict(X_future_revenue.iloc[-1:])
                
                # Обратное преобразование из логарифмической шкалы (exp(x)-1)
                quantity_pred = np.expm1(log_quantity_pred[0])
                revenue_pred = np.expm1(log_revenue_pred[0])

                quantity_pred = abs(quantity_pred)
                revenue_pred = abs(revenue_pred)
                
                # Сохранение прогноза после обратного преобразования
                forecast_quantity.append(quantity_pred)
                forecast_revenue.append(revenue_pred)

                forecast_quantity = [abs(q) for q in forecast_quantity]
                forecast_revenue = [abs(r) for r in forecast_revenue]
                
                # Обновление данных - добавляем как обычные, так и логарифмированные значения
                new_row = pd.DataFrame({
                    'quantity': [quantity_pred],
                    'final_price': [revenue_pred],
                    'quantity_log': [log_quantity_pred[0]],
                    'final_price_log': [log_revenue_pred[0]]
                }, index=[next_date])
                
                # Копируем все остальные признаки из последней строки
                for col in future_data.columns:
                    if col not in ['quantity', 'final_price', 'quantity_log', 'final_price_log']:
                        new_row[col] = future_data[col].iloc[-1]
                
                # Добавляем строку и удаляем старую
                future_data = pd.concat([future_data, new_row])
                if len(future_data) > 12:
                    future_data = future_data.iloc[1:]
            
            # Создание датафрейма прогноза
            forecast_df = pd.DataFrame({
                'forecast_quantity': [abs(q) for q in forecast_quantity],
                'forecast_revenue': [abs(r) for r in forecast_revenue],
                'quantity_lower': [abs(q * 0.9) for q in forecast_quantity],
                'quantity_upper': [abs(q * 1.1) for q in forecast_quantity],
                'revenue_lower': [abs(r * 0.9) for r in forecast_revenue],
                'revenue_upper': [abs(r * 1.1) for r in forecast_revenue]
            }, index=forecast_dates)
            
        except Exception as e:
            print(f"Ошибка при прогнозировании методом XGBoost: {e}")
            import traceback
            traceback.print_exc()
            return pd.DataFrame()

    
    # Прогнозирование средним значением
    elif method == 'average':
        try:
            # Используем среднее за последние периоды
            window = min(6, len(time_series))
            avg_quantity = time_series['quantity'].tail(window).mean()
            avg_revenue = time_series['final_price'].tail(window).mean()
            
            # Создание датафрейма прогноза
            forecast_df = pd.DataFrame({
                'forecast_quantity': [avg_quantity] * periods,
                'forecast_revenue': [avg_revenue] * periods,
                'quantity_lower': [avg_quantity * 0.8] * periods,  # примерный нижний интервал
                'quantity_upper': [avg_quantity * 1.2] * periods,  # примерный верхний интервал
                'revenue_lower': [avg_revenue * 0.8] * periods,    # примерный нижний интервал
                'revenue_upper': [avg_revenue * 1.2] * periods     # примерный верхний интервал
            }, index=forecast_dates)
            
        except Exception as e:
            print(f"Ошибка при прогнозировании методом среднего: {e}")
            return pd.DataFrame()
    
    else:
        print(f"Метод прогнозирования '{method}' не поддерживается.")
        print("Поддерживаемые методы: 'prophet', 'xgboost', 'average'")
        return pd.DataFrame()
    
    # Объединение исторических данных и прогноза
    try:
        result = forecast_df
        return result
        
    except Exception as e:
        print(f"Ошибка при объединении данных: {e}")
        return forecast_df


In [10]:

# ПРИМЕР ИСПОЛЬЗОВАНИЯ КОДА
def main():
    """
    Пример использования функций для анализа данных KUDO
    """
    try:
        # 1. Загрузка и консолидация данных
        print("1. Загрузка и консолидация данных...")
        consolidated_data = consolidate_orders('final_df_latest2.csv', 'denied.csv')
        print(f"Загружено записей: {len(consolidated_data)}")
        # 2. Расчет потенциальных продаж
        print("\n2. Расчет потенциальных продаж...")
        potential_sales = calculate_potential_sales(consolidated_data)
        print(f"Топ-5 SKU по потенциальной выручке:")
        print(potential_sales.sort_values('potential_revenue', ascending=False).head())
        potential_sales.sort_values('potential_revenue', ascending=False).head().to_csv("potential_sales.csv")
        # 3. Прогнозирование продаж
        print("\n3. Прогнозирование продаж...")
        # Общий прогноз
        time_series = prepare_time_series(consolidated_data)
        time_series[['quantity','final_price']].to_csv("quantity_and_revenue.csv")
        forecast = forecast_sales(time_series, periods=12,method ="xgboost")
        forecast.tail(12).to_csv("main_forecast.csv")
        print("Прогноз на следующие 12 месяцев:")
        print(forecast.tail(12))
        
        #Прогноз по категориям
        print("\nПрогнозирование по категориям продуктов...")
        category_forecasts = forecast_by_segment(consolidated_data, 'category', periods=12)
        for category, forecast in category_forecasts.items():
            print(f"\nПрогноз для категории {category}:")
            forecast.to_csv(f"forecast_for_{category}_category.csv")
            print(forecast.tail(12))
        
        # 4. Аналитические отчеты
        print("\n4. Аналитические отчеты по трендам...")
        # Отчеты по трендам
        trend_reports = generate_trend_report(
            consolidated_data, 
            dimensions=['category', 'group', 'region'], 
            metrics=['quantity', 'final_price']
        )
        
        print(f"Сгенерированы отчеты по трендам для {len(trend_reports)} измерений")
        
        # Топ клиентов и SKU
        top_clients = get_top_performers(consolidated_data, 'client_id', 'final_price')
        print("\nТоп-10 клиентов по выручке:")
        print(top_clients)
        top_clients.to_csv("top_clients.csv")
        
        top_skus = get_top_performers(consolidated_data, 'sku', 'quantity')
        print("\nТоп-10 SKU по количеству:")
        print(top_skus)
        top_skus.to_csv("top_skus.csv")
        print("\nАнализ из данных успешно завершен!")
        
    except Exception as e:
        print(f"Ошибка при выполнении анализа: {e}")

if __name__ == "__main__":
    main()


1. Загрузка и консолидация данных...
Загружено записей: 3079171

2. Расчет потенциальных продаж...
Топ-5 SKU по потенциальной выручке:
          sku  year  month  quarter  region      group   category  quantity  \
449064  18390  2023      9        3  москва  Пены KUDO  Пены KUDO    167228   
440303  18288  2023      9        3  москва  Пены KUDO  Пены KUDO    123427   
440204  18288  2023      7        3  москва  Пены KUDO  Пены KUDO    110232   
439789  18288  2022      9        3  москва  Пены KUDO  Пены KUDO     95506   
453512  18487  2021     10        4  москва  Пены KUDO  Пены KUDO    101259   

        actual_revenue       price  lost_quantity  lost_revenue  \
449064    5.696623e+07  351.706740            0.0           0.0   
440303    5.480913e+07  453.093842            0.0           0.0   
440204    4.482902e+07  418.399277            0.0           0.0   
439789    3.980481e+07  428.351792            0.0           0.0   
453512    3.887444e+07  393.327854            0.0      

19:51:29 - cmdstanpy - INFO - Chain [1] start processing
19:51:31 - cmdstanpy - INFO - Chain [1] done processing
19:51:31 - cmdstanpy - INFO - Chain [1] start processing
19:51:39 - cmdstanpy - INFO - Chain [1] done processing
19:51:40 - cmdstanpy - INFO - Chain [1] start processing


Недостаточно данных для сложных моделей. Используем метод prophet.


19:51:41 - cmdstanpy - INFO - Chain [1] done processing
19:51:41 - cmdstanpy - INFO - Chain [1] start processing
19:51:41 - cmdstanpy - INFO - Chain [1] done processing



Прогноз для категории KERRY®:
            forecast_quantity  forecast_revenue  quantity_lower  \
2024-07-01        662323.6250       110774208.0    596091.26250   
2024-08-01        667207.0625       106597792.0    600486.35625   
2024-09-01        659659.6250       111707984.0    593693.66250   
2024-10-01        661427.8750       106850000.0    595285.08750   
2024-11-01        659524.3750       106945016.0    593571.93750   
2024-12-01        659897.5000       106945016.0    593907.75000   
2025-01-01        657004.5625       106881800.0    591304.10625   
2025-02-01        615516.3125       103506096.0    553964.68125   
2025-03-01        642350.8125       101630024.0    578115.73125   
2025-04-01        642350.8125       106856112.0    578115.73125   
2025-05-01        650076.3750       106856112.0    585068.73750   
2025-06-01        654419.3750       106865088.0    588977.43750   

            quantity_upper  revenue_lower  revenue_upper  
2024-07-01    728555.98750     9969678